In [1]:
import os
import sys
import h5py
import ROOT

import numpy as np
import multiprocessing as mp

from tqdm import tqdm
from itertools import repeat

sys.path.append('..')
import utils_HDF5 as utils

delphes_path = '/usr/local/Delphes-3.4.2'
ROOT.gROOT.ProcessLine(f'.include {delphes_path}/')
ROOT.gROOT.ProcessLine(f'.include {delphes_path}/external/')
ROOT.gInterpreter.Declare(f'#include "{delphes_path}/classes/DelphesClasses.h"')
ROOT.gSystem.Load(f'{delphes_path}/install/lib/libDelphes')

MAX_JETS = 15
N_CORES = 64

Welcome to JupyROOT 6.20/08


In [2]:
def DeltaR(eta1, phi1, eta2, phi2):
    dEta = eta1 - eta2
    dPhi = np.abs(phi1 - phi2)

    dPhi = np.where(dPhi > np.pi, 2 * np.pi - dPhi, dPhi)

    dR = np.sqrt(dPhi**2 + dEta**2)
    return dR


def create_triHiggs_dataset(f, nevent, MAX_JETS):
    # with b-tagging information
    f.create_dataset('INPUTS/Source/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('INPUTS/Source/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('INPUTS/Source/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('INPUTS/Source/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('INPUTS/Source/mass', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('INPUTS/Source/btag', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')

    f.create_dataset('TARGETS/h1/b1', (nevent,), maxshape=(None,), dtype='<i8')
    f.create_dataset('TARGETS/h1/b2', (nevent,), maxshape=(None,), dtype='<i8')
    f.create_dataset('TARGETS/h2/b1', (nevent,), maxshape=(None,), dtype='<i8')
    f.create_dataset('TARGETS/h2/b2', (nevent,), maxshape=(None,), dtype='<i8')
    f.create_dataset('TARGETS/h3/b1', (nevent,), maxshape=(None,), dtype='<i8')
    f.create_dataset('TARGETS/h3/b2', (nevent,), maxshape=(None,), dtype='<i8')

    f.create_dataset('CLASSIFICATIONS/EVENT/signal', (nevent,), maxshape=(None,), dtype='<i8')


def write_dataset(file, data: list):
    nevent = len(data)

    for key in data[0].keys():
        # Resize
        shape = list(file[key].shape)
        shape[0] = nevent
        file[key].resize(shape)
        # Write
        value = np.array([data_dict[key] for data_dict in data])
        file[key][:] = value

In [3]:
def select_event(root_path, nbj_min, start, end):

    f = ROOT.TFile(root_path)
    tree = f.Get("Delphes")

    data_list = []
    for i in tqdm(range(start, end)):
        tree.GetEntry(i)

        # 夸克資料
        # b夸克 衰變前的編號
        quarks_id = []
        quarks_Eta = []
        quarks_Phi = []
        quarks_Jet = np.array([-1, -1, -1, -1, -1, -1])

        # 找出 3 個 final Higgs
        final_h_index = []
        for index, particle in enumerate(tree.Particle):
            if particle.PID == 25:
                h = index
                d1 = tree.Particle[h].D1
                while tree.Particle[d1].PID == 25:
                    h = d1
                    d1 = tree.Particle[h].D1
                final_h_index.append(h)

        final_h_index = list(set(final_h_index))

        # 找出 6 個 final b quark
        for h in final_h_index:
            # h > b b~
            b1 = tree.Particle[h].D1
            b2 = tree.Particle[h].D2

            # 找出 b 衰變前的編號
            d1 = tree.Particle[b1].D1

            while abs(tree.Particle[d1].PID) == 5:
                b1 = d1
                d1 = tree.Particle[b1].D1

            # 找出 b~ 衰變前的編號
            d2 = tree.Particle[b2].D1
            while abs(tree.Particle[d2].PID) == 5:
                b2 = d2
                d2 = tree.Particle[b2].D1

            quarks_id.extend([b1, b2])

        quarks_Eta.extend(tree.Particle[quark].Eta for quark in quarks_id)
        quarks_Phi.extend(tree.Particle[quark].Phi for quark in quarks_id)

        # 事件中的 jet 資料
        jet_PT = np.array([jet.PT for jet in tree.Jet])
        jet_Eta = np.array([jet.Eta for jet in tree.Jet])
        jet_Phi = np.array([jet.Phi for jet in tree.Jet])
        jet_Mass = np.array([jet.Mass for jet in tree.Jet])
        jet_BTag = np.array([jet.BTag for jet in tree.Jet])

        # Jet 資料
        # |eta| < 2.5 & PT > 20 GeV
        eta_pt_cut = np.array((np.abs(jet_Eta) < 2.5) & (jet_PT > 20))
        # |eta| < 2.5 & PT > 40 GeV
        eta_pt40_cut = np.array((np.abs(jet_Eta) < 2.5) & (jet_PT > 40))

        nj = eta_pt_cut.sum()

        # 至少要 6 jet
        if nj < 6:
            continue
        
        # 至少要 4 jet pT > 40 GeV
        if eta_pt40_cut.sum() < 4:
            continue
        nbj = np.array(jet_BTag[eta_pt_cut][:MAX_JETS]).sum()

        # 在前 MAX_JETS jets 中，至少要 nbj_min 個 b-jet
        if nbj < nbj_min:
            continue

        PT = np.array(jet_PT[eta_pt_cut])
        Eta = np.array(jet_Eta[eta_pt_cut])
        Phi = np.array(jet_Phi[eta_pt_cut])
        Mass = np.array(jet_Mass[eta_pt_cut])
        BTag = np.array(jet_BTag[eta_pt_cut])

        # 找出每個夸克配對的 jet
        # more_than_1_jet = False
        for quark in range(len(quarks_Jet)):
            dR = DeltaR(quarks_Eta[quark], quarks_Phi[quark], Eta, Phi)
            if dR.min() < 0.4:
                quarks_Jet[quark] = np.argmin(dR)


        quark_jet = quarks_Jet.reshape(1, 6)

        h1_mask = utils.get_particle_mask(quark_jet, [0, 1])
        h2_mask = utils.get_particle_mask(quark_jet, [2, 3])
        h3_mask = utils.get_particle_mask(quark_jet, [4, 5])

        # 準備寫入資料
        data_dict = {
            'INPUTS/Source/MASK': np.arange(MAX_JETS) < nj,
            'INPUTS/Source/pt': PT[:MAX_JETS] if nj > MAX_JETS else np.pad(PT, (0, MAX_JETS-nj)),
            'INPUTS/Source/eta': Eta[:MAX_JETS] if nj > MAX_JETS else np.pad(Eta, (0, MAX_JETS-nj)),
            'INPUTS/Source/phi': Phi[:MAX_JETS] if nj > MAX_JETS else np.pad(Phi, (0, MAX_JETS-nj)),
            'INPUTS/Source/mass': Mass[:MAX_JETS] if nj > MAX_JETS else np.pad(Mass, (0, MAX_JETS-nj)),
            'INPUTS/Source/btag': BTag[:MAX_JETS] if nj > MAX_JETS else np.pad(BTag, (0, MAX_JETS-nj)),

            'TARGETS/h1/b1': quarks_Jet[0] if h1_mask else -1,
            'TARGETS/h1/b2': quarks_Jet[1] if h1_mask else -1,
            'TARGETS/h2/b1': quarks_Jet[2] if h2_mask else -1,
            'TARGETS/h2/b2': quarks_Jet[3] if h2_mask else -1,
            'TARGETS/h3/b1': quarks_Jet[4] if h3_mask else -1,
            'TARGETS/h3/b2': quarks_Jet[5] if h3_mask else -1,

            'CLASSIFICATIONS/EVENT/signal': 1,

        }
        data_list.append(data_dict)
    return data_list

In [4]:
def from_root_to_h5(root_path, output_path, nbj_min=0):
    # root_path: input root file path
    # output_path: output h5 file path
    # nbj_min: 最少要有幾個 b-jet

    f = ROOT.TFile(root_path)
    nevent = f.Get("Delphes").GetEntries()
    print(f'Number of events: {nevent}')

    # Multi-core processing
    print(f'Number of cores: {N_CORES}')
    start = [nevent // N_CORES * i for i in range(N_CORES)]
    end = [nevent // N_CORES * (i+1) for i in range(N_CORES)]
    end[-1] = nevent

    with mp.Pool(processes=N_CORES) as pool:
        results = pool.starmap(select_event, zip(repeat(root_path), repeat(nbj_min), start, end))
    data_list = [data_dict for result_list in results for data_dict in result_list]

    # write to h5 file
    with h5py.File(output_path, 'w') as f_out:
        create_triHiggs_dataset(f_out, nevent, MAX_JETS)
        write_dataset(f_out, data_list)

In [ ]:
root_path = './MG5/data/gghhh_bsm_500_275/delphes_events_20241206_005920.root'
output_path = './SPANet/sig/gghhh-4pT40_4b_20241206_005920'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

In [10]:
root_path = './MG5/gghhh_bsm/Events/run_01_decayed_1/tag_2_delphes_events.root'
output_path = './SPANet/gghhh_4b_PT40_new.h5'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

Number of events: 100000
Number of cores: 64


100%|██████████| 1562/1562 [00:15<00:00, 103.99it/s]

100%|██████████| 1562/1562 [00:18<00:00, 85.78it/s] 


In [ ]:
root_path = './MG5/gghhh_bsm/Events/run_01_decayed_1/tag_2_delphes_events.root'
output_path = './SPANet/gghhh_4b_PT40_new.h5'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

In [9]:
root_path = '/home/public/3h_1500_1000.root'
output_path = './SPANet/gghhh_4b_PT40_1500_1000.h5'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

Number of events: 100000
Number of cores: 64


100%|██████████| 1562/1562 [00:18<00:00, 82.27it/s] 
Error in <THashList::Delete>: A list is accessing an object (0x5569f3a87480) already deleted (list name = THashList)
Error in <TList::Delete>: A list is accessing an object (0x5569f40edab0) already deleted (list name = TList)
Error in <THashList::Delete>: A list is accessing an object (0x5569f3a87480) already deleted (list name = THashList)
Error in <TList::Delete>: A list is accessing an object (0x5569f45f2760) already deleted (list name = TList)
Error in <THashList::Delete>: A list is accessing an object (0x5569f3a87480) already deleted (list name = THashList)
Error in <TList::Delete>: A list is accessing an object (0x5569f53ec1b0) already deleted (list name = TList)
Error in <THashList::Delete>: A list is accessing an object (0x5569f3a87480) already deleted (list name = THashList)
Error in <TList::Delete>: A list is accessing an object (0x5569f4cc25a0) already deleted (list name = TList)
Error in <THashList::Delete>: A list is acc

In [10]:
root_path = '/home/public/3h_1500_850.root'
output_path = './SPANet/gghhh_4b_PT40_1500_850.h5'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

Number of events: 100000
Number of cores: 64


100%|██████████| 1562/1562 [00:19<00:00, 79.63it/s] 
Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo for version 2 of class GenParticle read from the file /home/public/3h_1500_850.root
   has a different checksum than the previously loaded StreamerInfo.
   Reading objects of type GenParticle from the file /home/public/3h_1500_850.root 
   (and potentially other files) might not work correctly.
   Most likely the version number of the class was not properly
   updated [See ClassDef(GenParticle,2)].
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float T; //number
vs
   float CtgTheta; //number
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float X; //number
vs
   float D0; //number
Warning in <TStreamerInfo::CompareContent>: 

In [11]:
root_path = '/home/public/3h_1300_1000.root'
output_path = './SPANet/gghhh_4b_PT40_1300_1000.h5'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

Number of events: 100000
Number of cores: 64


100%|██████████| 1562/1562 [00:18<00:00, 82.34it/s] 
Error in <TList::Clear>: A list is accessing an object (0x5569f44fdc40) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f3aa7a00) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f41c7400) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f4862170) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f48660e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f36904b0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f43ec890) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f43f33a0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5569f4504470) already del

In [6]:
root_path = './MG5/gghhh_bsm/Events/run_01_decayed_1/tag_1_delphes_events.root'
output_path = './SPANet/gghhh_0b_01.h5'
nbj_min = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {nbj_min}'
print(cmd)

python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_01_decayed_1/tag_1_delphes_events.root ./SPANet/gghhh_0b_01.h5 0


In [8]:
for i in range(2, 9):
    root_path = f'./MG5/gghhh_bsm/Events/run_{i:02}_decayed_1/tag_2_delphes_events.root'
    output_path = f'./SPANet/gghhh_6b_{i:02}.h5'
    nbj_min = 6

    cmd = f'python from_root_to_h5.py {root_path} {output_path} {nbj_min}'
    print(cmd)

python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_02_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_02.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_03_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_03.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_04_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_04.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_05_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_05.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_06_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_06.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_07_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_07.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm/Events/run_08_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_08.h5 6


In [9]:
rnds = [323, 423, 523, 614, 714]

for rnd in rnds:
    root_path = f'./MG5/gghhh_bsm_{rnd}/Events/run_01_decayed_1/tag_2_delphes_events.root'
    output_path = f'./SPANet/gghhh_6b_{rnd}.h5'
    nbj_min = 6

    cmd = f'python from_root_to_h5.py {root_path} {output_path} {nbj_min}'
    print(cmd)

python from_root_to_h5.py ./MG5/gghhh_bsm_323/Events/run_01_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_323.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm_423/Events/run_01_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_423.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm_523/Events/run_01_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_523.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm_614/Events/run_01_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_614.h5 6
python from_root_to_h5.py ./MG5/gghhh_bsm_714/Events/run_01_decayed_1/tag_2_delphes_events.root ./SPANet/gghhh_6b_714.h5 6


In [ ]:
# loop all files in the folder

for m3_m2 in [#'500_275', '500_300', '520_325'
              '570_250', 
              '600_325', '700_325', '800_325', '700_400', '800_400']:
    root_path = f'./MG5/data/gghhh_bsm_{m3_m2}/'
    output_path = f'./SPANet/sig/gghhh_bsm_{m3_m2}/'
    os.makedirs(output_path, exist_ok=True)
    nbj_min = 4
    for name in os.listdir(root_path):
        if name.endswith('.root'):
            root_file = os.path.join(root_path, name)
            output_file = os.path.join(output_path, name.replace('.root', '.h5'))
            cmd = f'python from_root_to_h5.py {root_file} {output_file} {nbj_min} &'
            print(cmd)

python from_root_to_h5.py ./MG5/data/gghhh_bsm_570_250/delphes_events_20241226_175330.root ./SPANet/sig/gghhh_bsm_570_250/delphes_events_20241226_175330.h5 4 &
python from_root_to_h5.py ./MG5/data/gghhh_bsm_570_250/delphes_events_20241218_150954.root ./SPANet/sig/gghhh_bsm_570_250/delphes_events_20241218_150954.h5 4 &
python from_root_to_h5.py ./MG5/data/gghhh_bsm_570_250/delphes_events_20241228_015103.root ./SPANet/sig/gghhh_bsm_570_250/delphes_events_20241228_015103.h5 4 &
python from_root_to_h5.py ./MG5/data/gghhh_bsm_570_250/delphes_events_20241227_095203.root ./SPANet/sig/gghhh_bsm_570_250/delphes_events_20241227_095203.h5 4 &
python from_root_to_h5.py ./MG5/data/gghhh_bsm_570_250/delphes_events_20241220_041321.root ./SPANet/sig/gghhh_bsm_570_250/delphes_events_20241220_041321.h5 4 &
python from_root_to_h5.py ./MG5/data/gghhh_bsm_600_325/delphes_events_20241220_041419.root ./SPANet/sig/gghhh_bsm_600_325/delphes_events_20241220_041419.h5 4 &
python from_root_to_h5.py ./MG5/data/ggh

# TRSM

In [6]:
root_path = './MG5/TRSM_420_280/Events/run_02_decayed_1/tag_1_delphes_events.root'
output_path = './SPANet/TRSM_420_280/gghhh_4b_PT40.h5'
nbj_min = 4

from_root_to_h5(root_path, output_path, nbj_min=nbj_min)

Number of events: 100000
Number of cores: 64


100%|██████████| 1562/1562 [00:18<00:00, 86.13it/s] 

100%|██████████| 1562/1562 [00:20<00:00, 74.47it/s] 
Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo for version 2 of class GenParticle read from the file ./MG5/TRSM_420_280/Events/run_02_decayed_1/tag_1_delphes_events.root
   has a different checksum than the previously loaded StreamerInfo.
   Reading objects of type GenParticle from the file ./MG5/TRSM_420_280/Events/run_02_decayed_1/tag_1_delphes_events.root 
   (and potentially other files) might not work correctly.
   Most likely the version number of the class was not properly
   updated [See ClassDef(GenParticle,2)].
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float T; //number
vs
   float CtgTheta; //number
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' diff

In [2]:
for m3, m2 in [(420, 280), (500, 275), (500, 300), (520, 325)]:
    root_path = f'./MG5/TRSM_{m3}_{m2}/Events/run_03_decayed_1/'
    output_path = f'./SPANet/TRSM/TRSM_{m3}_{m2}/'
    os.makedirs(output_path, exist_ok=True)
    nbj_min = 4

    name = 'tag_1_delphes_events.root'
    root_file = os.path.join(root_path, name)
    output_file = os.path.join(output_path, 'gghhh-4pT40_4b-03.h5')
    cmd = f'python from_root_to_h5.py {root_file} {output_file} {nbj_min} &'
    print(cmd)

python from_root_to_h5.py ./MG5/TRSM_420_280/Events/run_03_decayed_1/tag_1_delphes_events.root ./SPANet/TRSM/TRSM_420_280/gghhh-4pT40_4b-03.h5 4 &
python from_root_to_h5.py ./MG5/TRSM_500_275/Events/run_03_decayed_1/tag_1_delphes_events.root ./SPANet/TRSM/TRSM_500_275/gghhh-4pT40_4b-03.h5 4 &
python from_root_to_h5.py ./MG5/TRSM_500_300/Events/run_03_decayed_1/tag_1_delphes_events.root ./SPANet/TRSM/TRSM_500_300/gghhh-4pT40_4b-03.h5 4 &
python from_root_to_h5.py ./MG5/TRSM_520_325/Events/run_03_decayed_1/tag_1_delphes_events.root ./SPANet/TRSM/TRSM_520_325/gghhh-4pT40_4b-03.h5 4 &


In [8]:
# if __name__ == '__main__':

#     root_path = sys.argv[1]
#     output_path = sys.argv[2]
#     nbj_min = int(sys.argv[3])

#     main(root_path, output_path, nbj_min)